# Market depth (order book)


In [1]:
from ib_async import *

util.startLoop()

ib = IB()
ib.connect("127.0.0.1", 4001, clientId=16)

<IB connected to 127.0.0.1:4001 clientId=16>

To get a list of all exchanges that support market depth data and display the first five:


In [2]:
l = ib.reqMktDepthExchanges()
l[:5]

[DepthMktDataDescription(exchange='DTB', secType='OPT', listingExch='', serviceDataType='Deep', aggGroup=2147483647),
 DepthMktDataDescription(exchange='LSEETF', secType='STK', listingExch='', serviceDataType='Deep', aggGroup=2147483647),
 DepthMktDataDescription(exchange='SGX', secType='FUT', listingExch='', serviceDataType='Deep', aggGroup=2147483647),
 DepthMktDataDescription(exchange='IDEALPRO', secType='CASH', listingExch='', serviceDataType='Deep', aggGroup=4),
 DepthMktDataDescription(exchange='ARCA', secType='STK', listingExch='', serviceDataType='Deep', aggGroup=2147483647)]

Let's subscribe to market depth data for EURUSD:


In [2]:
# NQM2024 contract
contract = Contract(conId=620730920)
ib.qualifyContracts(contract)

ticker = ib.reqMktDepth(contract)

To see a live order book, an event handler for ticker updates is made that displays a dynamically updated dataframe:


In [ ]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(index=range(5), columns="bidSize bidPrice askPrice askSize".split())


def onTickerUpdate(ticker):
    bids = ticker.domBids
    for i in range(5):
        df.iloc[i, 0] = bids[i].size if i < len(bids) else 0
        df.iloc[i, 1] = bids[i].price if i < len(bids) else 0
    asks = ticker.domAsks
    for i in range(5):
        df.iloc[i, 2] = asks[i].price if i < len(asks) else 0
        df.iloc[i, 3] = asks[i].size if i < len(asks) else 0
    clear_output(wait=True)
    display(df)


ticker.updateEvent += onTickerUpdate

IB.sleep(15)

,bidSize,bidPrice,askPrice,askSize
0,1.0,17502.5,17503.25,2.0
1,3.0,17502.25,17503.5,5.0
2,5.0,17502.0,17503.75,5.0
3,8.0,17501.75,17504.0,9.0
4,4.0,17501.5,17504.25,2.0


Stop the market depth subscription:


In [4]:
ib.cancelMktDepth(contract)

In [5]:
ib.disconnect()